<a href="https://colab.research.google.com/github/Sanazzzmi/Sentiment_analysis_of_the_English_text_with_word_embedding_-_frequency_word/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import training data

In [ ]:
import pandas as pd
import csv
d = 200

df = pd.read_csv("Sentiment Analysis Dataset.csv", error_bad_lines=False, warn_bad_lines=False)
del df['ItemID']
del df['SentimentSource']
df.columns = ['sentiment','tweet']

df

,sentiment,tweet
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...
...,...,...
1578607,1,Zzzzzz.... Finally! Night tweeters!
1578608,1,"Zzzzzzz, sleep well people"
1578609,0,ZzzZzZzzzZ... wait no I have homework.
1578610,0,"ZzZzzzZZZZzzz meh, what am I doing up again?"


# preprocessing training data

In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

In [ ]:
import re

text = ' '.join(df['tweet'])
text = text.split()
freq_comm = pd.Series(text).value_counts()
rare = freq_comm[freq_comm.values == 1]

def get_clean_text(x):
    if type(x) is str:
        x = x.lower()
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        x = re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x) 
        #regex to remove to emails
        x = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x)
        #regex to remove URLs
        x = re.sub('RT', "", x)
        #substitute the 'RT' retweet tags with empty spaces
        x = re.sub('[^A-Z a-z]+', '', x)
        #combining all the text excluding rare words.
        x = ' '.join([t for t in x.split() if t not in rare])
        return x
    else:
        return x
    
df['tweet'] = df['tweet'].apply(lambda x: get_clean_text(x))
df['tweet']

0                                is so sad for my apl friend
1                              i missed the new moon trailer
2                                          omg its already o
3          omgaga im sooo im gunna cry i have been at thi...
4                         i think mi bf is cheating on me tt
                                 ...                        
1578607                        zzzzzz finally night tweeters
1578608                            zzzzzzz sleep well people
1578609                   zzzzzzzzzz wait no i have homework
1578610           zzzzzzzzzzzzz meh what am i doing up again
1578611                           zzzzzzzzzzzzzzzzzzz i wish
Name: tweet, Length: 1578612, dtype: object

In [ ]:
df['sentiment'].value_counts()

1    790177
0    788435
Name: sentiment, dtype: int64

In [ ]:
text = df['tweet'].tolist()
y = df['sentiment']

# Each word has an id


In [ ]:
from keras.preprocessing.text import Tokenizer
token = Tokenizer()
token.fit_on_texts(text)
#token.word_index.items()

In [ ]:
vocab_size  = len(token.word_index) + 1
vocab_size

642315

# Each sentence is written according to the number of the word

In [ ]:
encoded_text = token.texts_to_sequences(text)
print(encoded_text[:3])

[[5, 19, 105, 11, 7, 217244, 246], [1, 237, 3, 71, 763, 1232], [217, 66, 188, 419]]


In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_length = 120
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')
print(X)

[[     5     19    105 ...      0      0      0]
 [     1    237      3 ...      0      0      0]
 [   217     66    188 ...      0      0      0]
 ...
 [ 24800    133     39 ...      0      0      0]
 [ 48716   1985     52 ...      0      0      0]
 [134663      1    108 ...      0      0      0]]


# use word embedding glove

In [ ]:
import numpy as np
glove_vectors = dict()
# file = open('glove.twitter.27B.200d.txt', encoding='utf-8')
file = open('glove.twitter.27B.200d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    #storing the word in the variable
    vectors = np.asarray(values[1: ])
    #storing the vector representation of the respective word in the dictionary
    glove_vectors[word] = vectors
file.close()
len(glove_vectors)

1193515

In [ ]:
glove_vectors.get('you')

array(['0.1964', '0.67153', '0.0062976', '0.25359', '-0.42097', '0.3849',
       '1.0378', '-0.18536', '-0.054244', '-0.10856', '0.12146',
       '0.047692', '-0.93228', '-0.27192', '-0.3506', '0.11069',
       '-0.055099', '-0.079658', '0.011767', '0.14395', '-0.025917',
       '-0.18253', '0.025691', '0.19619', '0.10334', '1.0731', '0.41992',
       '0.17083', '0.91596', '-0.43398', '0.16464', '0.012715',
       '0.095477', '0.1449', '-0.29652', '-0.10642', '0.098389',
       '0.17937', '0.23289', '-0.22209', '0.08777', '-0.18005', '0.38678',
       '-0.032059', '0.1202', '-0.41741', '0.26827', '-0.19769',
       '0.021976', '-0.23585', '0.14789', '0.10173', '-0.1038',
       '-0.31954', '0.63308', '-0.051142', '-0.053209', '0.29378',
       '-0.054395', '4.3346e-05', '0.25585', '-0.18048', '-0.2703',
       '0.0077783', '0.63585', '0.14869', '-0.10854', '-0.24726',
       '-0.10869', '-0.24147', '0.28059', '0.13645', '-0.03561',
       '-0.68258', '0.11957', '0.011085', '-0.013771',

In [ ]:
glove_vectors.get('yogdfgfdgdgu')

#Find words in word embedding

In [ ]:
word_vector_matrix = np.zeros((vocab_size, d))


not_word = []
have_word = []

for word, index in token.word_index.items()  :# هر کلمه یه شناسه داره
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
        have_word.append(word)
    else:
        not_word.append(word)
    
#have_word =>Words in word embedding

#How many words are seen in positive sentences and how many in negative sentences

In [ ]:
#Divide tweets into tags of one and tags of zeros
df_y = pd.DataFrame(np.array(y),columns=['a'])
df_one = df_y[df_y['a']==1].index.values.astype(int)
df_zero = df_y[df_y['a']==0].index.values.astype(int)

text_one = []
for x in df_one:
    text_one.append(text[x])

text_zero = []
for x in df_zero:
    text_zero.append(text[x])

join_text_zero = [' '.join(text_zero)]
join_text_one = [' '.join(text_one)]

df_join_text_zero = pd.DataFrame(join_text_zero[0:],columns= ["tex"])
df_join_text_one = pd.DataFrame(join_text_one[0:],columns= ["tex"])

df_have_word = pd.DataFrame({'word':have_word})
df_token = pd.DataFrame.from_dict(token.word_index.items())

df_text_one = pd.DataFrame(text_one[0:],columns= ["tex"])
df_text_zero = pd.DataFrame(text_zero[0:],columns= ["tex"])

from collections import Counter
count_zero = df_join_text_zero['tex'].str.split().apply(Counter)
count_one = df_join_text_one['tex'].str.split().apply(Counter)

df_counter_zero = pd.DataFrame.from_dict(count_zero[0], orient='index').reset_index()
df_counter_one = pd.DataFrame.from_dict(count_one[0], orient='index').reset_index()
df_counter_zero.columns = ['word', 'value']
df_counter_one.columns = ['word', 'value']

df_token.columns = ['word', 'id']

sentiment_zero =df_have_word.merge(df_counter_zero, left_on='word', right_on='word')[['value','word']]
sentiment_one = df_have_word.merge(df_counter_one, left_on='word', right_on='word')[['value','word']]

#Words found in both positive and negative sentences
common = sentiment_one.merge(sentiment_zero,on=['word','word'])
common.columns = ['value_one', 'word' , 'value_zero']

common["max"] = common[["value_one", "value_zero"]].max(axis=1)
common["min"] = common[["value_one", "value_zero"]].min(axis=1)

#Words that were both positive and negative in the sentence were marked with a label column that were more commonly seen in positive or negative sentences.
common["lable"] = np.where(common["value_one"] == common["max"], 1, 0)
common["frequent_negative_sentence"] = np.where(common["lable"] == 1 , common["min"], common["max"])
common["frequent_positive_sentence"] = np.where(common["lable"] == 0 , common["min"], common["max"])

common_df_final = common.drop('value_one', 1)
common_df_final = common_df_final.drop('value_zero', 1)
common_df_final = common_df_final.drop('max', 1)
common_df_final = common_df_final.drop('min', 1)

#Those words that are seen in positive sentences and not seen in negative sentences
just_one = sentiment_one[(~sentiment_one.word.isin(common.word))&(~sentiment_one.word.isin(common.word))]
just_one = just_one.assign(lable = 1)
just_one = just_one.assign(frequent_negative_sentence = 0)
just_one.columns = ['frequent_positive_sentence', 'word' , 'lable','frequent_negative_sentence']

#Those words that are seen in negative sentences and not seen in positive sentences
just_zero = sentiment_zero[(~sentiment_zero.word.isin(common.word))&(~sentiment_zero.word.isin(common.word))]
just_zero = just_zero.assign(lable = 0)
just_zero = just_zero.assign(frequent_posoitve_sentence = 0)
just_zero.columns = ['frequent_negative_sentence', 'word' , 'lable','frequent_positive_sentence']

frames = [common_df_final,just_zero, just_one]
result = pd.concat(frames)

result

,word,lable,frequent_negative_sentence,frequent_positive_sentence
0,i,0,556579,369736
1,to,0,310717,249686
2,the,1,255847,262505
3,a,1,179817,195778
4,is,0,186404,164141
...,...,...,...,...
83501,melantai,1,0,1
83502,kellies,1,0,1
83503,hanggat,1,0,1
83504,zaynah,1,0,1


# WI calculation

In [ ]:
result['WI_positive_word'] = np.where(result['lable']== 1 , 1- result['frequent_negative_sentence']/ result['frequent_positive_sentence'], None)
result['WI_negative_word'] = np.where(result['lable']== 0 , 1- result['frequent_positive_sentence']/ result['frequent_negative_sentence'], None)
result['WI'] = result['WI_positive_word'].fillna(result['WI_negative_word'])
result

,word,lable,frequent_negative_sentence,frequent_positive_sentence,WI_positive_word,WI_negative_word,WI
0,i,0,556579,369736,None,0.335699,0.335699
1,to,0,310717,249686,None,0.19642,0.196420
2,the,1,255847,262505,0.025363,None,0.025363
3,a,1,179817,195778,0.081526,None,0.081526
4,is,0,186404,164141,None,0.119434,0.119434
...,...,...,...,...,...,...,...
83501,melantai,1,0,1,1.0,None,1.000000
83502,kellies,1,0,1,1.0,None,1.000000
83503,hanggat,1,0,1,1.0,None,1.000000
83504,zaynah,1,0,1,1.0,None,1.000000


# Find words in the sentence that are in word embedding

In [ ]:
word_have_list= list(result.word)
word_have_set = set(word_have_list)
result_word_list = result['word'].tolist()
result_WI_list = result['WI'].tolist()

def word_WI_finder(x):
    df_words = set(x.split(' '))
    extract_words =  word_set.intersection(df_words)
    #index_word = result_word_list.index(extract_words)
    
    
    return (extract_words)


word_set = word_have_set

df['ddd_WI'] = df.tweet.apply(word_WI_finder)

# Calculate weight

In [ ]:
WI_ww = []
for x in df['ddd_WI'][:]:
    WI_w = []
    for y in x : 
        ind = result_word_list.index(y)
        WI_w.append(result_WI_list[ind])
    WI_ww.append(WI_w)
    
WI_ww_new = [[] if x==[0] else x for x in WI_ww]
WI_ww_neww = [[] if x==[0,0] else x for x in WI_ww_new]
Weight_sentence = [[float(j)/sum(i) for j in i] for i in WI_ww_neww[:]]

Weight_sentence[0]

[0.11417525396010611,
 0.04050639402398644,
 0.08692084970868079,
 0.33915257924121495,
 0.04535108576770854,
 0.05235419367581135,
 0.32153964362249177]

# Calculate the final sentence vector

In [ ]:
#part 1 :  start _ 789305

In [ ]:
sentence_ss1 = []
for i in range(len(df['ddd_WI'][0 : int(len(df)/2)])):
    #print(df['ddd_WI'][x])
    vv = []
    for j in range(len(df['ddd_WI'][i])):
        #print(list(df['ddd_WI'][i])[j])
        if len(Weight_sentence[i]) > 1:
            v = []
            for num in range(d):
                vec = Weight_sentence[i][j] * float(glove_vectors.get(list(df['ddd_WI'][i])[j])[num])
                v.append(vec)
            #print(v)
            vv.append(v)
        #print(vv)
        else:
            vv.append(glove_vectors.get(list(df['ddd_WI'][i])[j]).astype(np.float))
            
    s = [sum(x) for x in zip(*vv)]
    sentence_ss1.append(s)


data1 = pd.DataFrame(sentence_ss1[0:] , columns = list(range(d)))

data1[:].to_csv('X_train1.csv', sep='\t', encoding='utf-8')
df['sentiment'][0 : int(len(df)/2)].to_csv('y_train1.csv', sep='\t', encoding='utf-8')

In [ ]:
#part 2 : 789306 _ 1578612
sentence_ss2 = []
for i in range(int(len(df)/2) , int(len(df))):
    #print(df['ddd_WI'][x])
    vv = []
    for j in range(len(df['ddd_WI'][i])):
        #print(list(df['ddd_WI'][i])[j])
        if len(Weight_sentence[i]) > 1:
            v = []
            for num in range(d):
                vec = Weight_sentence[i][j] * float(glove_vectors.get(list(df['ddd_WI'][i])[j])[num])
                v.append(vec)
            #print(v)
            vv.append(v)
        #print(vv)
        else:
            vv.append(glove_vectors.get(list(df['ddd_WI'][i])[j]).astype(np.float))
            
    s = [sum(x) for x in zip(*vv)]
    sentence_ss2.append(s)


data2 = pd.DataFrame(sentence_ss2[0:] , columns = list(range(d)))

data2[:].to_csv('X_train2.csv', sep='\t', encoding='utf-8')
df['sentiment'][int(len(df)/2) : ].to_csv('y_train2.csv', sep='\t', encoding='utf-8')